In [1]:
import sys
sys.path.append('..')

from enum import Enum

from datetime import datetime, date, timedelta
from sqlalchemy import func, or_
import pandas as pd
import numpy as np

from src import crud
from src.models.enums.facebook.adset import Target
from src.database.session import SessionLocal
from src.models import *
from src.utils.common import print_dict
from src.utils.pd import eval_but_leave_string_if_you_cant
from src.pingers import *
from src.statistical_tests.bernoulli_tests.mean_test_bernoulli import *
from play.relevant_shops import *


db = SessionLocal()

shop_ = crud.shop.get(db=db, id=16038)

end_date = date.today()
start_date = end_date - timedelta(days=30)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2023-08-10 06:50:26.923 | DEBUG    | src.database.session:<module>:17 - base: prod.cw6tznt1p9hq.us-east-1.rds.amazonaws.com


In [4]:
shop_df = ping_target_and_performance(db=db, shop_id=16038, start_date='2023-07-01')

2023-08-10 06:51:20.492 | DEBUG    | src.pingers.ping_target:ping_target:68 -           audience interests
0   Audience.broad        []
1   Audience.broad        []
2   Audience.broad        []
3   Audience.broad        []
4   Audience.broad        []
..             ...       ...
57  Audience.broad        []
58  Audience.broad        []
59  Audience.broad        []
60  Audience.broad        []
61  Audience.broad        []

[62 rows x 2 columns]


In [5]:
descriptive_features = ["audience", "target", "gender"]
metric_columns = ["count_ads", "spend_USD", "impr", "link_clicks", "purch", 'spend']
group_cols = [
    "year_month",
    "feature",
    "feature_value",
]
descriptive_col = 'gender'

shop_df['count_ads'] = 1

In [34]:
from dateutil.relativedelta import relativedelta
date(year=2022, month=1, day=3) - relativedelta(months=5)

datetime.date(2021, 8, 3)

In [26]:
import os
os.path.join('/app/data/', 'data.csv')

'/data.csv'

In [23]:
for col_iter, descriptive_col in enumerate(descriptive_features):

    filtered = shop_df.loc[:, ["year_month", descriptive_col] + metric_columns].rename(columns={descriptive_col: "feature_value"})
    sum_by_month = filtered.groupby("year_month").sum()
    grouped = filtered.groupby(
        [
            "year_month",
            "feature_value",
        ]
    )

    absolute_sum = grouped.sum().loc[:, metric_columns]
    relative_sum = (absolute_sum / sum_by_month).rename(columns=lambda col_name: col_name + "_by_shop")

    full = absolute_sum.join(relative_sum)

    full["feature"] = descriptive_col
    full = full.set_index(["feature"], append=True)
    full = full.reorder_levels(group_cols, axis=0).fillna(0)

    # logger.debug(f"full index: {full.index.names}")
    # logger.debug(f"table_index: {table.index.names}")

    full["count_shops"] = (full.count_ads > 0).astype(int)

    if shop_iter == 0 and col_iter == 0 and from_scratch:
        table = full
    else:
        table = table.add(full, fill_value=0)

NameError: name 'shop_iter' is not defined

In [9]:
filtered = shop_df.loc[:, ["year_month", descriptive_col] + metric_columns].rename(
    columns={descriptive_col: "feature_value"}
)
filtered

,year_month,feature_value,count_ads,spend_USD,impr,link_clicks,purch,spend
0,2023-07-01,Gender.both,1,509.77,17945,402,88,509.77
1,2023-08-01,Gender.both,1,0.00,0,0,3,0.00
2,2023-07-01,Gender.female,1,2944.05,108632,415,172,2944.05
3,2023-08-01,Gender.female,1,1025.27,91510,890,11,1025.27
4,2023-07-01,Gender.female,1,21104.61,2014343,16658,131,21104.61
...,...,...,...,...,...,...,...,...
93,2023-08-01,Gender.both,1,1766.27,161713,2129,10,1766.27
94,2023-08-01,Gender.female,1,263.63,11601,66,14,263.63
95,2023-08-01,Gender.both,1,776.81,69587,1407,7,776.81
96,2023-08-01,Gender.both,1,1168.24,69637,515,12,1168.24


In [10]:
sum_by_month = filtered.groupby("year_month").sum()
sum_by_month

,count_ads,spend_USD,impr,link_clicks,purch,spend
year_month,,,,,,
2023-07-01,60,247202.34,15707007,159885,2384,247202.34
2023-08-01,38,73490.37,4229766,47957,847,73490.37


In [13]:
grouped = filtered.groupby(
    [
        "year_month",
        "feature_value",
    ]
)
grouped

In [14]:
absolute_sum = grouped.sum().loc[:, metric_columns]
absolute_sum

count_ads  spend_USD     impr  link_clicks  purch  \
year_month feature_value                                                      
2023-07-01 Gender.both           34   91899.39  6024562        78335    853   
           Gender.female         26  155302.95  9682445        81550   1531   
2023-08-01 Gender.both           25   41606.73  2695273        32278    328   
           Gender.female         13   31883.64  1534493        15679    519   

                              spend  
year_month feature_value             
2023-07-01 Gender.both     91899.39  
           Gender.female  155302.95  
2023-08-01 Gender.both     41606.73  
           Gender.female   31883.64

In [15]:
relative_sum = (absolute_sum / sum_by_month).rename(columns=lambda col_name: col_name + "_by_shop")
relative_sum

count_ads_by_shop  spend_USD_by_shop  impr_by_shop  \
year_month feature_value                                                       
2023-07-01 Gender.both             0.566667           0.371758      0.383559   
           Gender.female           0.433333           0.628242      0.616441   
2023-08-01 Gender.both             0.657895           0.566152      0.637216   
           Gender.female           0.342105           0.433848      0.362784   

                          link_clicks_by_shop  purch_by_shop  spend_by_shop  
year_month feature_value                                                     
2023-07-01 Gender.both               0.489946       0.357802       0.371758  
           Gender.female             0.510054       0.642198       0.628242  
2023-08-01 Gender.both               0.673061       0.387249       0.566152  
           Gender.female             0.326939       0.612751       0.433848

In [20]:
full = absolute_sum.join(relative_sum)
full["feature"] = descriptive_col
full = full.set_index(["feature"], append=True)
full = full.reorder_levels(group_cols, axis=0).fillna(0)
full["count_shops"] = (full.count_ads > 0).astype(int)
full

count_ads  spend_USD     impr  link_clicks  \
year_month feature feature_value                                               
2023-07-01 gender  Gender.both           34   91899.39  6024562        78335   
                   Gender.female         26  155302.95  9682445        81550   
2023-08-01 gender  Gender.both           25   41606.73  2695273        32278   
                   Gender.female         13   31883.64  1534493        15679   

                                  purch      spend  count_ads_by_shop  \
year_month feature feature_value                                        
2023-07-01 gender  Gender.both      853   91899.39           0.566667   
                   Gender.female   1531  155302.95           0.433333   
2023-08-01 gender  Gender.both      328   41606.73           0.657895   
                   Gender.female    519   31883.64           0.342105   

                                  spend_USD_by_shop  impr_by_shop  \
year_month feature feature_value                                    
2023-07-01 gender  Gender.both             0.371758      0.383559   
                   Gender.female           0.628242      0.616441   
2023-08-01 gender  Gender.both             0.566152      0.637216   
                   Gender.female           0.433848      0.362784   

                                  link_clicks_by_shop  purch_by_shop  \
year_month feature feature_value                                       
2023-07-01 gender  Gender.both               0.489946       0.357802   
                   Gender.female             0.510054       0.642198   
2023-08-01 gender  Gender.both               0.673061       0.387249   
                   Gender.female             0.326939       0.612751   

                                  spend_by_shop  count_shops  
year_month feature feature_value                              
2023-07-01 gender  Gender.both         0.371758            1  
                   Gender.female       0.628242            1  
2023-08-01 gender  Gender.both         0.566152            1  
                   Gender.female       0.433848            1

In [21]:
def get_new_summand(shop_df: pd.DataFrame, descriptive_col: str) -> pd.DataFrame:
    filtered = shop_df.loc[:, ["year_month", descriptive_col] + metric_columns].rename(
    columns={descriptive_col: "feature_value"}
    )
    sum_by_month = filtered.groupby("year_month").sum()
    grouped = filtered.groupby(
        (
            "year_month",
            "feature_value",
        )
    )

    absolute_sum_df = grouped.sum().loc[:, metric_columns]
    relative_sum_df = (absolute_sum_df / sum_by_month).rename(columns=lambda col_name: col_name + "_by_shop")

    result_df = absolute_sum_df.join(relative_sum_df)

    result_df["feature"] = descriptive_col
    result_df = result_df.set_index(["feature"], append=True)
    result_df = result_df.reorder_levels(group_cols, axis=0).fillna(0)

    result_df["count_shops"] = (result_df.count_ads > 0).astype(int)
    return result_df

In [ ]:
filtered = shop_df.loc[:, ["year_month", descriptive_col] + metric_columns].rename(
    columns={descriptive_col: "feature_value"}
)
sum_by_month = filtered.groupby("year_month").sum()
grouped = filtered.groupby(
    (
        "year_month",
        "feature_value",
    )
)

absolute_sum = grouped.sum().loc[:, metric_columns]
relative_sum = (absolute_sum / sum_by_month).rename(columns=lambda col_name: col_name + "_by_shop")

full = absolute_sum.join(relative_sum)

full["feature"] = descriptive_col
full = full.set_index(["feature"], append=True)
full = full.reorder_levels(group_cols, axis=0).fillna(0)

full["count_shops"] = (full.count_ads > 0).astype(int)